##### edited by Claire Valva, May 5, 2018

# 1979 to 2016 300hPa at 40.5 deg data processed
will be using file: 1979-2016-300hPa-40.5N-z.nc 
which is currently located in ~/uncategorized data

This file contains a single 2D array called “z”, whose dimension is (240, 55520).  The first dimension is longitude, 1.5 degree interval from 0 to 358.5 (0 is Greenwich Mean).  The second dimension is time, 6 hours apart, from 00 UTC January 1, 1979 to 18 UTC December 31, 2016.  The unit is meters^2/sec^2.  Divide by g = 9.81 m/s^2 to obtain geopotential height, the altitude at which you find 300 hPa.

In [1]:
#import packages, for now everything I think I'll need
import numpy as np
from netCDF4 import Dataset, num2date # This is to read .nc files and time array
import scipy.signal
from scipy.fftpack import fft, ifft, fftshift, fftfreq
import matplotlib.pyplot as plt
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import pandas as pd
import datetime
from math import pi

In [2]:
filepath = '/home/clairev/uncategorized-data/1979-2016-300hPa-40.5N-z.nc' # Location of the file
fileobj = Dataset(filepath, mode='r')

# Check what's in there
fileobj

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): longitude(240), time(55520)
    variables(dimensions): float32 z(time,longitude)
    groups: 

In [3]:
#load coordinates
height = fileobj.variables['z'][:]/9.81

#create time array
time_list = []
for i in range(0,55520):
    time_list.append(i*6)
tunit = "hours since 1979-01-01T00:00:00Z"
tarray = num2date(time_list,units = tunit,calendar = 'gregorian')
print(tarray)

#create longitude array
lon_list = []
for i in range(240):
    lon_list.append(i*1.5)

[datetime.datetime(1979, 1, 1, 0, 0) datetime.datetime(1979, 1, 1, 6, 0)
 datetime.datetime(1979, 1, 1, 12, 0) ...,
 datetime.datetime(2016, 12, 31, 6, 0)
 datetime.datetime(2016, 12, 31, 12, 0)
 datetime.datetime(2016, 12, 31, 18, 0)]


# plot 1st 3 months of geopotential height

In [23]:
#(for now) use scatterplot to make movie of 1st 3 months of geopotenital height
maxhgt = height[:].max()
minhgt = height[:].min()

for i in range(360):
    timestamp = tarray[i]
    
    plt.clf();
    plt.scatter(lon_list[:],height[i], color = "b")
    plt.ylim((minhgt, maxhgt))
    plt.xlim((0,365))
    plt.xlabel('longitude')
    plt.ylabel('geopotential height (m)')
    plt.title('geopotential height on '+str(timestamp))
    plt.tight_layout()
    plt.show

    plt.savefig('405N-processing-pic/geohgt'+str(i).zfill(3)+'.png')

# Hovmoeller diagram

In [28]:
# Setting the contour levels for the colored plot
color_intervals = np.linspace(minhgt * 0.99,
                              maxhgt*1.01, 60,
                              endpoint=True)

# Set the contour levels for the contour lines
contour_intervals = np.linspace(minhgt * 0.99,
                              maxhgt*1.01,
                                20,
                                endpoint=True)

In [39]:
#create list for plotting first few dates
firstdates = tarray[range(360)]
firsthgts = height[range(360)]
    
#create plot    
plt.clf();
plt.figure(figsize=(30, 30)); #a giant image but its detailed
plt.contourf(lon_list, firstdates, firsthgts, color_intervals, cmap='rainbow', alpha=0.5) 
plt.title('Hovmoeller Diagram')
plt.colorbar(shrink=0.5)
plt.contour(lon_list, firstdates, firsthgts, contour_intervals, colors='k', alpha=0.5)
plt.xlabel('longitude')
plt.ylabel('date')
plt.savefig('405N-processing-pic/hovtest.png')